# Clasificación de estrellas con redes neuronales feedforward

Clasificación de objetos (estrella, galaxia o quasar) utilizando datos científicos obtenidos del observatorio Apache Point de Nuevo México. El set de datos forma parte del proyecto Sloan Digital Sky Survey SDSS y contiene información de características espectrales.

Dirección del set de datos: https://www.kaggle.com/datasets/fedesoriano/stellar-classification-dataset-sdss17<br/>
Cantidad de registros: 100,000<br/>

Descripción de las columnas:

<table>
    <thead>
        <tr>
            <td>#</td>
            <td>Columna</td>
            <td>Descripción</td>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>1</td>
            <td>obj_ID</td>
            <td>Object Identifier, the unique value that identifies the object in the image catalog used by the CAS.</td>
        </tr>
        <tr>
            <td>2</td>
            <td>alpha</td>
            <td>Right Ascension angle (at J2000 epoch).</td>
        </tr>
        <tr>
            <td>3</td>
            <td>delta</td>
            <td>Declination angle (at J2000 epoch).</td>
        </tr>
        <tr>
            <td>4</td>
            <td>u</td>
            <td>Ultraviolet filter in the photometric system.</td>
        </tr>
        <tr>
            <td>5</td>
            <td>g</td>
            <td>Green filter in the photometric system.</td>
        </tr>
        <tr>
            <td>6</td>
            <td>r</td>
            <td>Red filter in the photometric system.</td>
        </tr>
        <tr>
            <td>7</td>
            <td>i</td>
            <td>Near Infrared filter in the photometric system.</td>
        </tr>
        <tr>
            <td>8</td>
            <td>z</td>
            <td>Infrared filter in the photometric system.</td>
        </tr>
        <tr>
            <td>9</td>
            <td>run_ID</td>
            <td>Run Number used to identify the specific scan.</td>
        </tr>
        <tr>
            <td>10</td>
            <td>rereun_ID</td>
            <td>Rerun Number to specify how the image was processed.</td>
        </tr>
        <tr>
            <td>11</td>
            <td>cam_col</td>
            <td>Camera column to identify the scanline within the run.</td>
        </tr>
        <tr>
            <td>12</td>
            <td>field_ID</td>
            <td>Field number to identify each field.</td>
        </tr>
        <tr>
            <td>13</td>
            <td>spec_obj_ID</td>
            <td>Unique ID used for optical spectroscopic objects (this means that 2 different observations with the same spec_obj_ID must share the output class).</td>
        </tr>
        <tr>
            <td>14</td>
            <td>class</td>
            <td>Object class (galaxy, star or quasar object).</td>
        </tr>
        <tr>
            <td>15</td>
            <td>redshift</td>
            <td>Redshift value based on the increase in wavelength.</td>
        </tr>
        <tr>
            <td>16</td>
            <td>plate</td>
            <td>Plate ID, identifies each plate in SDSS.</td>
        </tr>
        <tr>
            <td>17</td>
            <td>MJD</td>
            <td>Modified Julian Date, used to indicate when a given piece of SDSS data was taken.</td>
        </tr>
        <tr>
            <td>18</td>
            <td>fiber_ID</td>
            <td>Fiber ID that identifies the fiber that pointed the light at the focal plane in each observation.</td>
        </tr>
    </tbody>
</table>


# Librerías a utilizar en el proyecto

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LeakyReLU
from tensorflow.keras.optimizers import Adagrad
from tensorflow.keras.metrics import Recall, BinaryAccuracy, Precision
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.initializers import GlorotNormal

Evita imprimir varios logs al momento de entrenar el modelo con Keras

In [2]:
from tqdm.keras import TqdmCallback

# Análisis de datos

Cargando set de datos

In [3]:
star_classification = pd.read_csv('data1/star_classification.csv') 
star_classification.head()

,obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,spec_obj_ID,class,redshift,plate,MJD,fiber_ID
0,1.237661e+18,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,3606,301,2,79,6.543777e+18,GALAXY,0.634794,5812,56354,171
1,1.237665e+18,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,4518,301,5,119,1.176014e+19,GALAXY,0.779136,10445,58158,427
2,1.237661e+18,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,3606,301,2,120,5.152200e+18,GALAXY,0.644195,4576,55592,299
3,1.237663e+18,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,4192,301,3,214,1.030107e+19,GALAXY,0.932346,9149,58039,775
4,1.237680e+18,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,8102,301,3,137,6.891865e+18,GALAXY,0.116123,6121,56187,842


Según la descripción de los datos, las siguientes columnas no aportan valor para el entrenamiento:

<ul>
    <li>obj_ID</li>
    <li>run_ID</li>
    <li>rerun_ID</li>
    <li>cam_col</li>
    <li>field_ID</li>
    <li>spec_obj_ID</li>
    <li>plate</li>
    <li>MJD</li>
    <li>fiber_ID</li>
</ul>

In [4]:
star_classification = star_classification.drop(
    columns=['obj_ID', 'run_ID', 'rerun_ID', 'cam_col', 'field_ID', 'spec_obj_ID', 'plate', 'MJD', 'fiber_ID'])
star_classification.head()

,alpha,delta,u,g,r,i,z,class,redshift
0,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,GALAXY,0.634794
1,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,GALAXY,0.779136
2,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,GALAXY,0.644195
3,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,GALAXY,0.932346
4,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,GALAXY,0.116123


# Tratamiento de datos

One hot encoding para variable categórica

In [5]:
one_hot_encoding = pd.get_dummies(star_classification["class"], prefix='class')
star_classification = star_classification.drop(columns=['class'])
star_classification.head()

,alpha,delta,u,g,r,i,z,redshift
0,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,0.634794
1,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,0.779136
2,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,0.644195
3,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,0.932346
4,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,0.116123


Normalización de datos

In [6]:
mean = np.mean(star_classification, axis=0)
std = np.std(star_classification, axis=0)
star_classification = (star_classification - mean) / std
star_classification = pd.concat([star_classification, one_hot_encoding], axis=1)
star_classification.head()

,alpha,delta,u,g,r,i,z,redshift,class_GALAXY,class_QSO,class_STAR
0,-0.434604,0.425529,0.059755,0.054926,0.403962,0.046007,0.003937,0.079557,1,0,0
1,-0.339921,0.363402,0.088045,0.072456,1.584406,1.185097,0.092835,0.277096,1,0,0
2,-0.367251,0.582713,0.103327,0.067165,0.519745,0.150019,0.008808,0.092423,1,0,0
3,1.669523,-1.249105,0.004921,0.102210,1.059904,0.807610,0.018321,0.486770,1,0,0
4,1.737310,-0.150242,-0.080055,-0.092948,-1.697421,-1.767887,-0.098468,-0.630267,1,0,0


Verificación de valores nulos

In [7]:
star_classification.isna().sum(axis=0)

alpha           0
delta           0
u               0
g               0
r               0
i               0
z               0
redshift        0
class_GALAXY    0
class_QSO       0
class_STAR      0
dtype: int64

# Datos de entrenamiento y pruebas

Separación de datos para entrenamiento y validación

In [8]:
X_train_model, X_validation, y_train_model, y_validation = train_test_split(
    star_classification[["alpha", "delta", "u", "g", "r", "i", "z", "redshift"]], 
    star_classification[["class_GALAXY", "class_QSO", "class_STAR"]],
    test_size=0.8, random_state=2022)

Separación de datos para entrenamiento

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    X_train_model, 
    y_train_model, 
    test_size=0.8, random_state=2022)

# Modelo Keras

## Optimizador

In [10]:
adagrad = Adagrad( learning_rate=0.01,
    initial_accumulator_value=0.1,
    epsilon=1e-07,
    name="Adagrad"
)

## Custom callback

In [11]:
class custom_callback(Callback):
    def __init__(self):
        self.f1score = 0
    
    def get_f1score(self):
        return self.f1score
    
    def set_f1score(self, recall, precision):
        denominador = recall + precision
        if( denominador == 0):
            self.f1score = 0
        else:
            self.f1score = (2*(recall * precision) / denominador)
    
    def on_epoch_end(self, batch, logs=None):
        recall = logs.get('recall')
        accuracy = logs.get('accuracy')
        precision = logs.get('precision')
        self.set_f1score(recall, precision)

## Modelo secuencial con sigmoid

In [15]:
model_sigmoid = Sequential()
model_sigmoid.add(Dense(4, input_shape=(8,), activation="sigmoid", kernel_initializer=GlorotNormal()))
model_sigmoid.add(Dropout(0.1, input_shape=(8,)))
model_sigmoid.add(Dense(8, activation="sigmoid"))
model_sigmoid.add(Dropout(0.1, input_shape=(6,)))
model_sigmoid.add(Dense(6, activation="sigmoid"))
model_sigmoid.add(Dropout(0.1, input_shape=(3,)))
model_sigmoid.add(Dense(3, activation="softmax"))

# train the model using SGD
callbacks_sigmoid = custom_callback()
model_sigmoid.compile(
    loss="categorical_crossentropy", 
    optimizer=adagrad, 
    metrics=[Recall(name="recall"), BinaryAccuracy(name="accuracy"), Precision(name="precision")])

modelo_prediccion_sigmoid = model_sigmoid.fit(
    X_train, 
    y_train, 
    validation_data=(X_test, y_test), 
    epochs=20,
    batch_size=128, 
    verbose=0, 
    callbacks=[callbacks_sigmoid, TqdmCallback(verbose=1)])

print("F1-Score", callbacks_sigmoid.get_f1score())

0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

F1-Score 0.5499865097066087


## Modelo secuencial con relu

In [16]:
model_relu = Sequential()
model_relu.add(Dense(4, input_shape=(8,), activation="relu", kernel_initializer=GlorotNormal()))
model_relu.add(Dropout(0.1, input_shape=(8,)))
model_relu.add(Dense(8, activation="relu"))
model_relu.add(Dropout(0.1, input_shape=(6,)))
model_relu.add(Dense(6, activation="relu"))
model_relu.add(Dropout(0.1, input_shape=(3,)))
model_relu.add(Dense(3, activation="softmax"))

# train the model using SGD
print("[INFO] training network...")
callbacks_relu = custom_callback()
model_relu.compile(
    loss="categorical_crossentropy", 
    optimizer=adagrad, 
    metrics=[Recall(name="recall"), BinaryAccuracy(name="accuracy"), Precision(name="precision")])

modelo_prediccion_relu = model_sigmoid.fit(
    X_train, 
    y_train, 
    validation_data=(X_test, y_test), 
    epochs=20,
    batch_size=128, 
    verbose=0, 
    callbacks=[callbacks_relu, TqdmCallback(verbose=1)])

print("F1-Score", callbacks_relu.get_f1score())

[INFO] training network...


0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

F1-Score 0.5614035183155163


## Modelo secuencial con Leaky Relu

In [17]:
model_leakyrelu = Sequential()
model_leakyrelu.add(Dense(4, input_shape=(8,), activation=LeakyReLU(alpha=0.01), kernel_initializer=GlorotNormal()))
model_leakyrelu.add(Dropout(0.1, input_shape=(8,)))
model_leakyrelu.add(Dense(8, activation=LeakyReLU(alpha=0.01)))
model_leakyrelu.add(Dropout(0.1, input_shape=(6,)))
model_leakyrelu.add(Dense(6, activation=LeakyReLU(alpha=0.01)))
model_leakyrelu.add(Dropout(0.1, input_shape=(3,)))
model_leakyrelu.add(Dense(3, activation="softmax"))

# train the model using SGD
print("[INFO] training network...")
callbacks_leakyrelu = custom_callback()
model_leakyrelu.compile(
    loss="categorical_crossentropy", 
    optimizer=adagrad, 
    metrics=[Recall(name="recall"), BinaryAccuracy(name="accuracy"), Precision(name="precision")])

modelo_prediccion_leakyrelu = model_sigmoid.fit(
    X_train, 
    y_train, 
    validation_data=(X_test, y_test), 
    epochs=20,
    batch_size=128, 
    verbose=0, 
    callbacks=[callbacks_leakyrelu, TqdmCallback(verbose=1)])

print("F1-Score", callbacks_leakyrelu.get_f1score())

[INFO] training network...


0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]

F1-Score 0.5597768432238618
